<a href="https://colab.research.google.com/github/sipocz/pid_time_series/blob/main/ABB_to_CSV_220919.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os

In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
user = "SL"
uzem = "Szint1"

In [5]:
if user=="Anna":
    path = "/content/drive/MyDrive/Egyetem_MSc/Diplomamunka/2022Anna/Data/Abb/Sp/"
else:
    #path = "/content/drive/MyDrive/2022Anna/Data/Abb/Szteroid2/április/"

    #Üzemenkénti futtatáshoz:

    #path = "/content/drive/MyDrive/2022Anna/Data/Abb/Nor/"
    #path = "/content/drive/MyDrive/2022Anna/Data/Abb/Sp/"            #Külön kód van rá, legalul
    path = "/content/drive/MyDrive/2022Anna/Data/Abb/Szint1/"
    #path = "/content/drive/MyDrive/2022Anna/Data/Abb/Szint3/"
    #path = "/content/drive/MyDrive/2022Anna/Data/Abb/Szteroid2/"      #KÉSZ


In [12]:
#Listát készít a driveon lévő mappa exceleinek elérési útvonaláról (debug módban kiirja az elérési utakat)
def create_file_list(path_name, extension=".xlsx",debug=False):
 
  dir_path = os.path.dirname(path_name)
  eleresi_utak = []

  for root, dirs, files in os.walk(dir_path):
    for file in files:
      if file.endswith(extension):
        if debug:
          print(root+'/'+str(file))
        eleresi_utak.append(root+'/'+str(file))
  return (eleresi_utak)

In [13]:
#File lista meghívás
file_list=create_file_list(path, ".xlsx")

In [14]:
file_list

['/content/drive/MyDrive/2022Anna/Data/Abb/Szint1/június/431_TC8_202261.xlsx',
 '/content/drive/MyDrive/2022Anna/Data/Abb/Szint1/június/DW_AX4_TIC_202261.xlsx',
 '/content/drive/MyDrive/2022Anna/Data/Abb/Szint1/június/DW_AX4_PIC48_202261.xlsx',
 '/content/drive/MyDrive/2022Anna/Data/Abb/Szint1/június/DW_AX4_FIC_202261.xlsx',
 '/content/drive/MyDrive/2022Anna/Data/Abb/Szint1/június/DW_AX4_PIC03_202261.xlsx',
 '/content/drive/MyDrive/2022Anna/Data/Abb/Szint1/június/DW_AX1_PIC45_202261.xlsx',
 '/content/drive/MyDrive/2022Anna/Data/Abb/Szint1/június/DW_AX1_FIC_202261.xlsx',
 '/content/drive/MyDrive/2022Anna/Data/Abb/Szint1/június/DW_AX1_PIC03_202261.xlsx',
 '/content/drive/MyDrive/2022Anna/Data/Abb/Szint1/június/431_TC8_202262.xlsx',
 '/content/drive/MyDrive/2022Anna/Data/Abb/Szint1/június/DW_AX4_TIC_202262.xlsx',
 '/content/drive/MyDrive/2022Anna/Data/Abb/Szint1/június/DW_AX4_PIC48_202262.xlsx',
 '/content/drive/MyDrive/2022Anna/Data/Abb/Szint1/június/DW_AX4_FIC_202262.xlsx',


In [15]:
print (len(file_list))

482


In [16]:
#A megadott DataFrame-et 1 másodperces időközökre alakítja és interpolálja
def interpolate_sheet(df):
    df.columns = ["Date","Value","Status"]

    bad_data=df[df["Status"]=="Bad Data"].index
    df.drop(index=bad_data,inplace=True)
     
    df.index=df.Date
    df_vissza = df.resample("1s").mean()
    df_vissza['Value'].interpolate('linear',inplace = True)
    return(df_vissza)

In [17]:
#Az excel minden munkalapját interpolálja, majd ezeket összefűzi és kimenti a drive-ra .csv-ben
#Sp-re nem működik!! Arra külön kód

#Teszt Lacinak
#for i in range(2):

#Ez az éles futtatáshoz
for i,file_name in enumerate(file_list):
  
  #urlname = file_list[i]
  urlname = file_name
  _,fname = os.path.split(urlname)

  df=pd.read_excel(urlname,sheet_name=0)
  df_A = interpolate_sheet(df)

  df=pd.read_excel(urlname,sheet_name=1)
  df_Out = interpolate_sheet(df)

  df=pd.read_excel(urlname,sheet_name=2)
  df_Sp = interpolate_sheet(df)

  df=pd.read_excel(urlname,sheet_name=3)
  df_Pv = interpolate_sheet(df)

  df=pd.read_excel(urlname,sheet_name=4)
  df_ISp = interpolate_sheet(df)

  df_new = pd.concat([df_A, df_Out, df_Sp, df_Pv, df_ISp], axis=1)
  df_new.columns = ["A","Out","Sp","Pv","ISp"]
  
  if user=="Anna":
    out_file_path="/content/drive/MyDrive/Egyetem_MSc/Diplomamunka/2022Anna/Datapipeline/3/"
  else:
    out_file_path="/content/drive/MyDrive/2022Anna/Datapipeline/2/"
  
    
  out_file_name = out_file_path + fname[0:-5] + '.csv'
  print(i,"--",out_file_name)
  df_new.to_csv(out_file_name)

0 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/431_TC8_202261.csv
1 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/DW_AX4_TIC_202261.csv
2 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/DW_AX4_PIC48_202261.csv
3 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/DW_AX4_FIC_202261.csv
4 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/DW_AX4_PIC03_202261.csv
5 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/DW_AX1_PIC45_202261.csv
6 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/DW_AX1_FIC_202261.csv
7 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/DW_AX1_PIC03_202261.csv
8 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/431_TC8_202262.csv
9 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/DW_AX4_TIC_202262.csv
10 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/DW_AX4_PIC48_202262.csv
11 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/DW_AX4_FIC_202262.csv
12 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/DW_AX4_PIC03_202262.csv
13 -- /content/drive/MyDrive/

In [18]:
if user=="Anna":
    path = "/content/drive/MyDrive/Egyetem_MSc/Diplomamunka/2022Anna/Data/Abb/Sp/"
else:
    #path = "/content/drive/MyDrive/2022Anna/Data/Abb/Szteroid2/április/"

    #Üzemenkénti futtatáshoz:

    #path = "/content/drive/MyDrive/2022Anna/Data/Abb/Nor/"
    path = "/content/drive/MyDrive/2022Anna/Data/Abb/Sp/"            #Külön kód van rá, legalul
    #path = "/content/drive/MyDrive/2022Anna/Data/Abb/Szint1/"
    #path = "/content/drive/MyDrive/2022Anna/Data/Abb/Szint3/"
    #path = "/content/drive/MyDrive/2022Anna/Data/Abb/Szteroid2/"      #KÉSZ


In [19]:
file_list=create_file_list(path, ".xlsx")


In [20]:
print(len(file_list))

939


In [ ]:
#Sp üzem kódja

#Teszt Lacinak
#for i in range(2):

#Ez az éles futtatáshoz
for i,file_name in enumerate(file_list):
  
  urlname = file_list[i]
  #urlname = file_name
  _,fname = os.path.split(urlname)

  df=pd.read_excel(urlname,sheet_name=0)
  df.drop(df.tail(1).index,inplace=True)
  df_A = interpolate_sheet(df)

  df=pd.read_excel(urlname,sheet_name=1)
  df.drop(df.tail(1).index,inplace=True)
  df_Out = interpolate_sheet(df)

  df=pd.read_excel(urlname,sheet_name=2)
  df.drop(df.tail(1).index,inplace=True)
  df_Sp = interpolate_sheet(df)

  df=pd.read_excel(urlname,sheet_name=3)
  df.drop(df.tail(1).index,inplace=True)
  df_Pv = interpolate_sheet(df)

  df=pd.read_excel(urlname,sheet_name=4)
  df.drop(df.tail(1).index,inplace=True)
  df_ISp = interpolate_sheet(df)

  df_new = pd.concat([df_A, df_Out, df_Sp, df_Pv, df_ISp], axis=1)
  df_new.columns = ["A","Out","Sp","Pv","ISp"]
  
  if user=="Anna":
    out_file_path="/content/drive/MyDrive/Egyetem_MSc/Diplomamunka/2022Anna/Datapipeline/3/"
  else:
    out_file_path="/content/drive/MyDrive/2022Anna/Datapipeline/2/"


  out_file_name = out_file_path + fname[0:-5] + '.csv'
  print(i,"--",out_file_name)
  df_new.to_csv(out_file_name)

0 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/72C03_TC_202271.csv
1 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/72C03_SC_202271.csv
2 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/77D01_TC1_202271.csv
3 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/77C02_SC_202271.csv
4 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/77C02_TC_202271.csv
5 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/74C04_SC_202271.csv
6 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/74C03_SC_202271.csv
7 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/89D01_TC1_202271.csv
8 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/89D01_TC8_202271.csv
9 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/89D01_SC_202271.csv
10 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/72C03_TC_202272.csv
11 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/72C03_SC_202272.csv
12 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/77D01_TC1_202272.csv
13 -- /content/drive/MyDrive/2022Anna/Datapipeline/2/77C0